In [31]:
from db.client import PGClient
from collections import ChainMap
from datetime import datetime
import re
import emoji
import pandas as pd
client = PGClient()

In [102]:

s = 'ya 😄'
emojis_list = map(lambda x: ''.join(x.split()), emoji.UNICODE_EMOJI.keys())
r = re.compile('|'.join(re.escape(p) for p in emojis_list))
print(''.join(r.sub(' ',s)))

ya  


In [101]:
r.sub('x','ya 😄')

'ya x'

In [132]:
uni_emojis_list = map(lambda x: ''.join(x.split()), emoji.UNICODE_EMOJI.keys())
uni_emojis_pt = re.compile('|'.join(re.escape(p) for p in uni_emojis_list))

def clean(c):
    url_pt = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
    custom_emoji_pt = '<:\w+:[0-9]+>'
    tag_pt = '<@!?[0-9]+>'
   
    c = str(c)
    c = re.sub(url_pt, '', c)
    c = re.sub(tag_pt, '', c)
    c = re.sub('\n',' ', c)
    c = uni_emojis_pt.sub('', c)
    c = re.sub('"','', c)
    
    emojis = re.findall(custom_emoji_pt,c)
    for e in emojis:
        e_new = e.split(':')[1].lower()
        #c = re.sub(e, ' XXX'+e_new+'XXX ', c)
        c = re.sub(e, '', c)
    
    
    return c.strip()

In [135]:
q_messages = """SELECT json_build_object('id',id,'posted_at',posted_at,'content',content, 'author_name', author_name) FROM messages 
                WHERE author_id NOT IN (155149108183695360, 349547918966915073, 159985870458322944)
                ORDER BY id
                """ 

cursor = client.query(q_messages)
message = cursor.fetchone()
discussions = [[]]
previous_message = datetime.strptime(message[0]['posted_at'], '%Y-%m-%dT%H:%M:%S')

while message:
    message = message[0]
    content = clean(message['content'])
    if content:
        message['content'] = content
        message_date = datetime.strptime(message['posted_at'], '%Y-%m-%dT%H:%M:%S')
        time_diff = (message_date - previous_message).total_seconds() / 60
        if(time_diff > 60):
            discussions.append([message])
            previous_message = message_date

        else:
            discussions[-1:][0].append(message)
    message = cursor.fetchone()
    #print(message)

In [122]:
discussions[221]

[{'author_name': '【l】【e】【v】',
  'content': 'forgot to',
  'id': 350619978078355456,
  'posted_at': '2017-08-25T12:38:36'},
 {'author_name': '【l】【e】【v】',
  'content': 'open them',
  'id': 350619981463158797,
  'posted_at': '2017-08-25T12:38:37'},
 {'author_name': '【l】【e】【v】',
  'content': 'also',
  'id': 350620352059277313,
  'posted_at': '2017-08-25T12:40:06'},
 {'author_name': '【l】【e】【v】',
  'content': 'turned to mix',
  'id': 350620362142384129,
  'posted_at': '2017-08-25T12:40:08'},
 {'author_name': '【l】【e】【v】',
  'content': 'cuz of ping',
  'id': 350620370480398337,
  'posted_at': '2017-08-25T12:40:10'},
 {'author_name': '【l】【e】【v】',
  'content': 'fuck that system',
  'id': 350620381742104577,
  'posted_at': '2017-08-25T12:40:13'},
 {'author_name': 'GreyPit',
  'content': 'Isnt it because of fps lag?',
  'id': 350620541935157249,
  'posted_at': '2017-08-25T12:40:51'},
 {'author_name': 'kjx',
  'content': 'lel slabijan rage',
  'id': 350621664591085569,
  'posted_at': '2017-08-25T12

In [137]:
d_contents = []
for d in discussions:
    if len(d) > 250:
        content = ''
        for message in d:
            content += ' ' + message['content']
        d_contents.append(content)
df = pd.DataFrame(d_contents)
df.to_clipboard(index=False, header=False)

In [18]:
sorted_arr = sorted(arr, key=len, reverse=True)

In [19]:
sorted_arr[1]

IndexError: list index out of range

In [11]:
sorted_arr[1]
for d in sorted_arr[1]:
    print(d['content'])

IndexError: list index out of range

In [12]:
import numpy as np
discussions = np.array(discussions)
discussions.head()

AttributeError: 'numpy.ndarray' object has no attribute 'head'

In [9]:
import pandas as pd
discussions_df = pd.DataFrame.from_dict(discussions)

ValueError: arrays must all be same length

In [16]:
for c in max_d:
    print(c['content'])

<@156052059555233792>  dont forget to wake me up!!!
I SAID IT MY WAY !
and <@!242569944821268481>
WHY AM I SCREAMING
i DO have audacity: https://i.gyazo.com/233bbc94d714ce7ee3439105a8e36c7b.png
Shut up Hitler
iznodolf
dude dont call me hitler man
<:what:357486247746207745>
Hitler did nothing wrong
not funny bro
Dave: dude dont call me hitler man
wtf
Send shirtless pics
<:kappa:348186810117259265>
https://www.youtube.com/watch?v=AH_klyogSUE <@99247855344025600>
I'm sorry Dav, I'm afraid I can't do that
dave* ffs

shirtless pic
np
<@227488711477166081> <:pogchamp:389392356933107722>
<:kappapride:348190713285967872>
u jebemti
too small <:feelsbadman:348185002065788938>
<:notlikethis:348908822313566209>
would catch a grenade for ya
https://www.youtube.com/watch?v=7qnd-hdmgfk
<:lul:352045627917598720>
Go more down dave
<:swastika:309699814680166400>
<:swastika:309699814680166400>
<:swastika:309699814680166400>
<:kappa:348186810117259265>
i have nipples as penises so
<:what:35748624774620774

In [1]:
from paralleldots import set_api_key, get_api_key, sentiment, similarity

In [2]:
set_api_key("iQVOCSTVfHzRd67ybpvqaEWEgCRQefqWuIQfRypVm0I") 


In [149]:
s = "and there's nothing to change didn't used my lvl 80 against any`1. simple. and yet this conversation doesn't change anything about you being a fake friend all from me kek i tried not making a clan out of JH to prevent drama did it just make more drama? What drama? fucking all this fucking shit basem use VPN germany and dont talk no drama then :gg: >dont talk >problem solved сука блять иди нахуй пизда or that ah limited nice picture writing the truth about some1 isn't a drama *sigh* *unzips* here we go again are you playing emunow or will you be playing emulater yes it's sincerity I mean things which You can see up [5:23 PM] Trelacki: You will go cry again so what's the problem  /// not if it will be justified, i want you to get a higher lvl, but not for the sake of community wat. radek priest radek priest [5:24 PM] Trelacki: didn't used my lvl 80 against any`1.  /// you didnt, but you did say a guy is a terrorist just becausse he is from iraq which a high lvl admin shouldnt say so i need to know for god's sake trela, pls. am i getting demoted or no? Why not I'm allowed to say whatever I wnat want* that's called freedom of speech or You are like social media, You disallow this XD well, thats the reason you arent getting higher lvl, admins on jh are supposed to help people and not calling them something they might not be, whcih could be insulting to them also I didn't use word terrorist another lie iraq guy asked for help, i helped him and then you asked me why am i helping a terrorist it was vissible in cod2 channel and clay saw it hm. maybe. i believe the logic behind this is if you are higher level than usual (60+), you are suppose to be nice to people and helping them, etc. not being racist OR hurting their feelings. because if you are 60+ you are trusted with a server control therefore there are people who are trusting you. help him plant the bomb in czech republic! i was never and i never will be against you XDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDD but i also dont want innocent people who want to jump to be abused Europe balances the right to free expression with the right to be free of offense, meaning your right to speak is limited by speech that I might reasonably take as offensive. if he is innocent You're not the 1 who caused You to be 200kg+ k XD OH MY GOD GUYS KEEP GOING PLEASE IM ENJOYING THIS SO MUCH np inb4 thats what she said CMON I GOR GOT do you have crusti croc? basem finished real_autist now its your turn okay im going to have dinner but when im back, i want there to still be drama ok? honestly. radek. NO OFFEENSE PLEASE I STILL LOVE YOU.  but you shouldn't be 90 IMHO because you are being mean to basem etc. which is clearly not allowed deal cu deal fu I take offense of terorrist coming to server on which I want peace basem is also mean to me sometimes and he knwos im joking and speaking some unknown language mixed with letters and numbers how do you know they are terrorists? fucking special code to detonate bomb or what what if there are other tunisian,iraq, etc. people on server do you think they will take it as a joke? I fignore them lol and what do they do to offend you? they exist? they believe in killing people? you cant know what a random guy believes in if he is from iraq, he is muslim if he is muslim, he believe in killing people Trela hell no fuck trela i really need AN OFFICIAL statement thats because all christians wait till marriage to have sex. of what you just said maybe if he is member of ISIS limited go to orban and tell him they are cool and you want 1 in your house You know nothign about orban ill go to fucking orban and tell him i want one in my server k xDDDD Orban hates them Quran Quran (2:191-193) - And kill them wherever you find them, and turn them out from where they have turned you out. And Al-Fitnah [disbelief or unrest] is worse than killing... but if they desist, then lo! Allah is forgiving and merciful. And fight them until there is no more Fitnah [disbelief and worshipping of others along with Allah] and worship is for Allah alone. But if they cease, let there be no transgression except against Az-Zalimun(the polytheists, and wrong-doers, etc.) (Translation is from the Noble Quran) The verse prior to this (190) refers to fighting for the cause of Allah those who fight you leading some to claim that the entire passage refers to a defensive war in which Muslims are defending their homes and families.  The historical context of this passage is not defensive warfare, however, since Muhammad and his Muslims had just relocated to Medina and were not under attack by their Meccan adversaries.  In fact, the verses urge offensive warfare, in that Muslims are to drive Meccans out of their own city (which they later did).  Verse 190 thus means to fight those who offer resistance to Allah's rule (ie. Muslim conquest).  The use of the word persecution by some Muslim translators is disingenuous - the actual Arabic words for persecution (idtihad) - and oppression are not used instead of fitna.  Fitna can mean disbelief, or the disorder that results from unbelief or temptation.  A strict translation is 'sedition,' meaning rebellion against authority (the authority being Allah).  This is certainly what is meant in this context since the violence is explicitly commissioned until religion is for Allah - ie. unbelievers desist in their unbelief.  [Editor's note: these notes have been modified slightly after a critic misinterpreted our language. Verse 193 plainly says that 'fighting' is sanctioned even if the fitna 'ceases'.  This is about religious order, not real persecution.] have fun hitler was atheist. ANd a good painter i dont need to quote his ideology here i hope. ye, and You love them limited, so go for a conversation with him you're basically quoting bible verses here trela. Who said I love the terrorists? which are about as bad as quran ones there is a big space between hating and loving trela. I think Im out No point of this argument izno never shared his information what if hes a muslim? Trela is full of hate, so no point you still talk to him i believe those words are out of context..real muslims are friendly and not monsters..but ye this is what media presents us trela has no way of knowing my religion. read this shit book claay then simple. Quran (2:191-193) Have u ever read it? Quran (2:191-193) Quran (2:191-193) Quran (2:191-193) google it yes 3 verses thats all u got GO READ IT THEN TALK TO ME the worst guy from lebanon that i know is a fucking christian GET SOME FKN AKNOWLEDGE the Islamic sacred book, believed to be the word of God as dictated to Muhammad by the archangel Gabriel and written down in Arabic. the Islamic sacred book, believed to be the word of God as dictated to Muhammad by the archangel Gabriel and written down in Arabic. ABOUT THIS DISEASE do you believe in god trela? Hard to not belive in yourself lol. believe* trela u didnt notes that u talk too much about muslims and say some shit about them ? i am serious do you believe in the christian god trela? do you believe in real God. christian god yes. Using those types of stories without context would be like saying that the United States is a racist/genocidal country because they bombed Japan in 1945, killing 250,000 people. The statement that they bombed Japan is true, but there's a whole context (a preceding global war that lasted 6 years) that it needs to be considered in the context of. how can I believe in 1 person which was walking on the water Izno please that was his son you dont even know christian religion how can you judge muslims so you dont believe in christian god BUT you believe in Quran which is basically an islam bible cause I don't care about this religion I don't believe in this, these retards believe that's why they kill people GEEZOS one min son of god ? wtf lol basem, as a muslim, do you belive in killing people? as a muslim k ISIS presents ISLAM in their way . and thats what u see thats what media wants us to believe. thats not even islam its disgreace im giving up on trela. he really wants to see this stuff his way, and there is no way to change his mind, no matter how much evidence to the contrary you present him. and right now he'll be convinced that we are all pro-islam becuase we sound anti-trela you see it right Izno ^ pls islam word mean security, safety, peace, belly, safeness yes, tell that to storm izno, he wanted to change him as well That's why they rape european girls for their look cause they look like bitches better to look like a fucking ninja as said, i give up. because white ppl dont do that at all right? google again I'll send You here and read how many of those happened by muslims in last 2 years in europe found it WHITE NINJA And do you know why are the muslims coming? Arab spring my friend don't call me your friend cuz You're nobody for me atm thats why you still capitalize the You cmon guys i see u all talking about smth u never know anything about it you* right Izno. I don't respect him . xD Okay, the friend here doesnt mean a real friend k ignored. good solution always An adult doesnt ingoring a problem, he solves it And idk why you feel as an attack against youxd that's the first google search result for religion of peace ok , now i have to explain what is the ISLAM and what is the MUSLIM Go I want to hear Im courious Islam is a religion that believes in one God I(Allah) A Muslim is someone who follows or practices Islam, a monotheistic Abrahamic religion. done. if i had half a year then id submit a map for contest (c) pauli !poll delay contest more? yes 2020 delay for 1 month they wearing that white shit cuz their habits , and am muslim as they are and i didnt do what they do or eat what they eat or like what they like or feel like they feel !poll wait tona map BaSeM: they wearing that white shit cuz their habits , and am muslim as they are and i didnt do what they do or eat what they eat or like what they like or feel like they feel reminds me of rajo told me if I tell this to him he will don't angry for me so thats not really mean all muslims are good , or all muslims are bad in everything there is good & bad yes even kubi has good inside of him k , kubi switch to real life and move on. how about kubi and claay who is better by the behavior of those people which I saw IRL I can argue about it till You change Your opinion. everyone will say claay kubi and kubi too is not bad but he is smth like autistic xdd go finish real autist u see , u cant say anything now Tona - Today at 5:36 PM  basem finished real_autist now its your turn the only thing we can do here is just live in ur life to move on and dont talk 2 eachother ? and ignore everything is not for u dont mention any muslim/islam shits or better. dont mention religion in JH kee p your thoughts for yourself. good You just call him an autist without reason there is exactly what claay do so everyone like claay he is quite intelligent person. and you call all muslims terrorists without reasons because they are. tbh if he has 200+ ppl on fignore list (fignoring without reason) there is something wrong with him. then why hasnt basem exploded yet. aha , back to this shit lev his bomb lagged Don't You consider he don't want to have constant spam on chat / don't want to talk with people at all? cg_chattime 0 so that's why he is actively ignoring people for months on end through all possible channels Reasons. and he does not ignore czech people interesting. Nothing happens without a reason. he even ignored my voice chat through grey's stream he was watching only responded to shit when grey repeated it... ye maybe but he is do what u do always call all muslims are terrorists just and trela if you trust google google this are all muslims terorrists i think everyone know what is islam but what is islam does mean i think here noone know !en basem islam the best word ever ye islam = everything good ye everything will be good when instead of europa will be eurabia muslim <= islam eh shit why u wont understand If You're going to some country as a refugee You just submit to those people to which You are oging going* accept them and live with them in peace there is in everything good and bad You don't want them to change everything on your favor simple. thats all i can say Basem please... just end up this conversation.. I want to take rest after work gn :D Not going to slepe. sleep* ah k remember this when u talking about islam or muslims there is in everything good and bad there is in everything good and bad there is in everything good and bad you repeating this sentence makes me believe theres everything bad and bad like your time management what do you mean i cant work on map cuz iznohund doesnt reply me lol some one added me on fb and tell me there is JH on 1.0 sounds like raul no looks like tunisian he can talk EN & FR & AR i believe theres no jh on 1.0 i believe so too. lol he is tunisian ask him to say number cmon 10 years old i think this kid omg lol in 10 years he will be dead cuz of suicide bomb attack that's what I can see by his face xd thanks for not breaking the deal guys there is still drama trela joking i think just to enjoy u ye trela clearly joking about this what is your opinion on this matter just like kubi is trolling you dont want my opinion on this we want things going to happen why else would i ask for it kubi = trela  trela = kubi  } both.... tunisian math loooooooool kjx If (kubi=trela) make.people.bomb; pls xDD u know kjx++ xDD i forgot to declare the variable only 3 humans on this planet know kjx++ nah it looks okay to me i hope u learn this trela how many animals ? 0 kjx compiler reports success this code unfisnihed does nothing limited pls ik Just having fun"
sentiment(s)

ConnectionError: ('Connection aborted.', BadStatusLine('<html>\r\n',))

In [152]:
sentiment('hello from the other side you idiotic master hello from the other side you idiotic master')

{'probabilities': {'negative': 0.548038,
  'neutral': 0.351081,
  'positive': 0.100881},
 'sentiment': 'negative',
 'usage': 'By accessing ParallelDots API or using information generated by ParallelDots API, you are agreeing to be bound by the ParallelDots API Terms of Use: http://www.paralleldots.com/terms-and-conditions'}

In [154]:
sentiment('hello from the other side you idiotic master hello from the other side you idiotic master hello from the other side you idiotic master hello from the other side you idiotic master')

{'probabilities': {'negative': 0.425193,
  'neutral': 0.360317,
  'positive': 0.214491},
 'sentiment': 'negative',
 'usage': 'By accessing ParallelDots API or using information generated by ParallelDots API, you are agreeing to be bound by the ParallelDots API Terms of Use: http://www.paralleldots.com/terms-and-conditions'}